In [1]:
# importing the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import os 

In [2]:
# Dataset path
data_dir = "/content/drive/My Drive/catsvsdogs"

In [3]:
os.listdir(data_dir)

['Test', 'Train']

In [4]:
Train_dir = data_dir + "/Train/"
Test_dir = data_dir + "/Test/"

In [5]:
os.listdir(Train_dir)

['Dog', 'Cat']

In [6]:
os.listdir(Test_dir)

['Dog', 'Cat']

In [56]:
dog_path = Train_dir + "/Dog/"
dog_test = Test_dir + "/Dog/"

In [57]:
cat_path = Train_dir + "/Cat/"
cat_test = Test_dir + "/Cat/"

In [19]:
cat_path+os.listdir(cat_path)[6576]

'/content/drive/My Drive/catsvsdogs/Train//Cat/666.jpg'

In [20]:
# checking a cat sample details including file size represented by st_size
os.stat(cat_path+os.listdir(cat_path)[6576])

os.stat_result(st_mode=33152, st_ino=6624, st_dev=82, st_nlink=1, st_uid=0, st_gid=0, st_size=0, st_atime=1592937154, st_mtime=1592937148, st_ctime=1592937148)

In [21]:
# checking file size 
os.stat(cat_path+os.listdir(cat_path)[6576]).st_size

0

In [22]:
# remove file with 0 byte
os.remove(cat_path+os.listdir(cat_path)[6576])

In [47]:
# removing files less than 5 kb from  the dog  training dataset
count = 0
for image_filename in os.listdir(dog_path):
  img = dog_path + image_filename
  size = (os.stat(img).st_size)/1000
  if size < 5:
    count += 1
    os.remove(img)   
print(count)

148


In [48]:
# removing files less than 5 kb from  the cat  training dataset
count = 0
for image_filename in os.listdir(cat_path):
  img = cat_path + image_filename
  size = (os.stat(img).st_size)/1000
  if size < 5:
    count += 1
    os.remove(img)
print(count)

152


In [59]:
# removing files less than 5 kb from  the dog  test dataset
count = 0
for image_filename in os.listdir(dog_test):
  img = dog_test + image_filename
  size = (os.stat(img).st_size)/1000
  if size < 5:
    count += 1
    os.remove(img)   
print(count)

34


In [60]:
# removing files less than 5 kb from  the cat  test dataset
count = 0
for image_filename in os.listdir(cat_test):
  img = cat_test + image_filename
  size = (os.stat(img).st_size)/1000
  if size < 5:
    count += 1
    os.remove(img)
print(count)

35


In [49]:
#!pip install piexif

In [23]:
#import piexif
#x=0
#for image_filename in os.listdir(Train_dir + "/Dog"):
#  img = Train_dir + "/Dog/" + image_filename
#  try:
#    piexif.remove(img)
#  except:
#    x+=1
#    os.remove(img)

#print(x)

In [24]:
#import piexif
#x=0
#for image_filename in os.listdir(Train_dir + "/Cat"):
#  img = Train_dir + "/Cat/" + image_filename
#  try:
#    piexif.remove(img)
#  except:
#    x+=1
#    os.remove(img)

#print(x)

In [8]:
# checking data dimension or shape to select an average shape for the total dataset 
#dim1 = []
#dim2 = []

#for image_filename in os.listdir(Train_dir + "/Dog"):
  
#  img = imread(Train_dir + "/Dog/" + image_filename)
#  height, width, color = img.shape
#  dim1.append(width)
#  dim2.append(height)

In [9]:
#np.mean(dim1)

In [10]:
#np.mean(dim2)

In [50]:
# the mean dimension of the dataset is large so we resize

img_shape = (70,70,3)

In [51]:
# data augmentation and preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [52]:
image_gen = ImageDataGenerator(rescale=1./255,rotation_range=10,zoom_range=0.2)

In [53]:
print(Train_dir)
print(Test_dir)

/content/drive/My Drive/catsvsdogs/Train/
/content/drive/My Drive/catsvsdogs/Test/


In [54]:
# directing the data generator to the dataset path which must include sub-directories for the class of data to be trained 
image_gen.flow_from_directory(Train_dir)  

Found 19614 images belonging to 2 classes.


In [61]:
image_gen.flow_from_directory(Test_dir)

Found 4936 images belonging to 2 classes.


In [62]:
# import sequential model
from tensorflow.keras.models import Sequential

In [63]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten

In [64]:
# building the model architecture
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=img_shape,activation="relu" ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(4,4),input_shape=img_shape,activation="relu" ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(4,4),input_shape=img_shape,activation="relu" ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [65]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 67, 67, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       131200    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [66]:
# setting up early stopping to prevent overfitting 
from tensorflow.keras.callbacks import EarlyStopping

In [67]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)

In [68]:
batch_size = 16

In [69]:
# preprocess dataset using data generator and direct its path using flow from directory method
train_image_gen = image_gen.flow_from_directory(Train_dir,
                                                target_size=img_shape[:2],
                                                color_mode='rgb',
                                                batch_size=batch_size,
                                                class_mode='binary')

Found 19614 images belonging to 2 classes.


In [70]:
test_image_gen = image_gen.flow_from_directory(Test_dir,
                                                target_size=img_shape[:2],
                                                color_mode='rgb',
                                                batch_size=batch_size,
                                                class_mode='binary',
                                                shuffle=False)

Found 4936 images belonging to 2 classes.


In [71]:
# checking classes attached to the dataset
train_image_gen.class_indices

{'Cat': 0, 'Dog': 1}

In [72]:
# Fitting a model to the dataset 
result = model.fit(train_image_gen,epochs=40,
                             validation_data=test_image_gen,
                             callbacks=[early_stop])

Epoch 1/40
1226/1226 [==============================] - 16917s 14s/step - loss: 0.6714 - accuracy: 0.5800 - val_loss: 0.6467 - val_accuracy: 0.6319
Epoch 2/40
1226/1226 [==============================] - 350s 286ms/step - loss: 0.6101 - accuracy: 0.6691 - val_loss: 0.5679 - val_accuracy: 0.7046
Epoch 3/40
1226/1226 [==============================] - 342s 279ms/step - loss: 0.5498 - accuracy: 0.7205 - val_loss: 0.5205 - val_accuracy: 0.7445
Epoch 4/40
1226/1226 [==============================] - 350s 286ms/step - loss: 0.5024 - accuracy: 0.7574 - val_loss: 0.4814 - val_accuracy: 0.7634
Epoch 5/40
1226/1226 [==============================] - 351s 286ms/step - loss: 0.4715 - accuracy: 0.7759 - val_loss: 0.4448 - val_accuracy: 0.7857
Epoch 6/40
1226/1226 [==============================] - 346s 282ms/step - loss: 0.4354 - accuracy: 0.7996 - val_loss: 0.4476 - val_accuracy: 0.7881
Epoch 7/40
1226/1226 [==============================] - 350s 286ms/step - loss: 0.4176 - accuracy: 0.8107 - val_

In [75]:
# model performance 
pd.DataFrame(model.history.history)

,loss,accuracy,val_loss,val_accuracy
0,0.671401,0.579994,0.646656,0.631888
1,0.610120,0.669114,0.567862,0.704619
2,0.549835,0.720506,0.520512,0.744530
3,0.502409,0.757418,0.481365,0.763371
4,0.471504,0.775925,0.444829,0.785656
5,0.435366,0.799582,0.447590,0.788088
6,0.417599,0.810696,0.413982,0.813209
7,0.393022,0.822168,0.415421,0.805916
8,0.376287,0.832008,0.396747,0.822731
9,0.362089,0.840471,0.389387,0.828809


In [76]:
model.save("/content/drive/My Drive/catsvsdogs")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/catsvsdogs/assets


In [82]:
# save trianed model
model.save_weights("img_classification.h5")

In [83]:
model.save("catsvsdogs.h5")

In [84]:
from tensorflow.keras.models import load_model

In [86]:
# load trained model
model = load_model("catsvsdogs.h5")

In [87]:
# check model summary to be sure model was correctly loaded 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 67, 67, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 33, 33, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       131200    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [89]:
# check model evaluation
model.evaluate(test_image_gen)

309/309 [==============================] - 40s 129ms/step - loss: 0.4229 - accuracy: 0.8156


[0.4229377806186676, 0.8156402111053467]

In [90]:
# check model prediction which will return class probability
pred = model.predict(test_image_gen)

In [95]:
predictions = pred > 0.5

In [96]:
predictions

array([[False],
       [False],
       [ True],
       ...,
       [ True],
       [False],
       [ True]])

In [91]:
from sklearn.metrics import classification_report,confusion_matrix

In [92]:
test_image_gen.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [97]:
print(classification_report(test_image_gen.classes, predictions))

              precision    recall  f1-score   support

           0       0.77      0.90      0.83      2468
           1       0.88      0.72      0.79      2468

    accuracy                           0.81      4936
   macro avg       0.82      0.81      0.81      4936
weighted avg       0.82      0.81      0.81      4936



In [98]:
print(confusion_matrix(test_image_gen.classes,predictions))

[[2226  242]
 [ 681 1787]]


In [103]:
# making predictions with our model

dog_sample = dog_test + os.listdir(dog_test)[10]
dog_sample

'/content/drive/My Drive/catsvsdogs/Test//Dog/10147.jpg'

In [112]:
# to be sure of our sample

from tensorflow.keras.preprocessing import image

# load and resize the sample image
test_sample = image.load_img(dog_sample, target_size=(70,70))

# scale the image
img_array = image.img_to_array(test_sample)/255

# bcos our model expect batches of images, we reshape our image to (1,70,70,3) format
img_array = np.expand_dims(img_array, axis = 0)
img_array.shape

(1, 70, 70, 3)

In [113]:
# since we have preprocessed our sample image, its time to predict

model.predict(img_array)

array([[0.59030205]], dtype=float32)

In [114]:
# Recall our class indices

test_image_gen.class_indices

{'Cat': 0, 'Dog': 1}